# Define model to convert (I used a different notebook to keep code clean)

In [1]:
%run Functions.ipynb

# Load Model

In [2]:
net = Darknet('yolo_v3.cfg')
net.load_weights('yolov3.weights')

# Set model into evaluation mode

In [3]:
net.eval()
print()

# Prepare input (for converting and verifying ONNX model output)

In [4]:
x = torch.randn(4, 3, 128, 64, requires_grad=True)
torch_out = net(x)

blocks: 


# Convert the model into ONNX

In [5]:
# Export the model
torch.onnx.export(net,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "YOLO_V3.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

blocks: 


# Load exported ONNX model

In [6]:
import onnx

onnx_model = onnx.load("YOLO_V3.onnx")
onnx.checker.check_model(onnx_model)

# Verify ONNX output

In [7]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("YOLO_V3.onnx")

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out[0]['x']), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

Exported model has been tested with ONNXRuntime, and the result looks good!


# Convert output back into dictionary

#### The following function returns the ONNX output to the original Pytorch output.
#### This is due to the original output being a dictionary, which is incompatible with the conversion to ONNX.

In [8]:
def ONNX_output_to_original(ort_outs):

    results = {}
    for idx in range(0,len(ort_outs),3):
        counter = int(idx/3)
        results[counter]={'x':ort_outs[idx], 'a':ort_outs[idx+1], 'n':ort_outs[idx+2]}
        
    return results

In [9]:
results = ONNX_output_to_original(ort_outs)